<a href="https://colab.research.google.com/github/SreyaSalil/IR-Assignments/blob/main/IR_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IR Assignment 4**

*Implementation of Probablistic model*

## Import Packages

In [1]:
import string
import itertools
import math
import operator
# To calculate mean
from statistics import mean
# To access files
import os,sys
#upload files to Colab
from google.colab import files
# To use datatables
import pandas as pd
# To strip closed and unclosed HTML tags
from bs4 import BeautifulSoup
#To remove Numbers in text using RE
import re
#accent removal
import unicodedata
#stop word removal
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
#Stemming
from nltk.stem import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Preprocessing Functions for Query


### Lexical Analysis

In [2]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        new_word = re.sub(r'\d+','',word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def lexical_analysis(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_numbers(words)
    return words

### Stop word elimination

In [3]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    stop_words = set(stopwords.words("english"))
    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return new_words

### Stemming

In [4]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

### Input Inverted Index

In [5]:
uploaded = files.upload()

Saving inverted_index.xlsx to inverted_index.xlsx


In [6]:
df = pd.read_excel("inverted_index.xlsx",index_col="Unnamed: 0")
inverted_index = df.copy()
df

,T1.txt,T10.txt,T2.txt,T3.txt,T4.txt,T5.txt,T6.txt,T7.txt,T8.txt,T9.txt
abandon,0,2,0,0,0,0,0,0,0,1
abbrevy,0,0,0,0,0,0,0,1,0,0
abdom,0,0,0,0,0,0,0,1,0,0
abdomin,0,0,0,0,0,0,0,1,0,0
aberdeen,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
zint,0,0,2,0,0,0,0,0,0,12
zip,0,1,0,0,0,0,0,0,0,0
zon,0,0,0,0,0,0,0,2,0,0
zoolog,5,0,0,0,0,0,1,2,0,0


### Query Preprocessing Function

In [7]:
def preprocess_query(query):
    sample = query
    sample = sample.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    tokens = word_tokenize(sample)
    lexical = lexical_analysis(tokens)
    filtered_tokens = remove_stopwords(lexical)
    stemmed_tokens = stem_words(filtered_tokens)
    filtered_tokens1 = remove_stopwords(stemmed_tokens)
    return filtered_tokens1

## Ranking documents for a given query using Probabilistic Model

### Accepting and Preprocessing Query

In [8]:
query = input("Enter the query : ")
query = strip_html(query)
query_words = preprocess_query(query)
qw = set(query_words)

Enter the query : lung-fish


### Calculate Nw and (N-Nw)/Nw values

In [13]:
N = df.shape[1]
nw = df.astype(bool).sum(axis=1)
prob = (N-nw+0.5)/(nw+0.5)
print(N)
print("Nw values")
print(nw)
print("Probability values")
print(prob)

10
Nw values
abandon     2
abbrevy     1
abdom       1
abdomin     1
aberdeen    1
           ..
zint        2
zip         1
zon         1
zoolog      3
zoophyt     2
Length: 3772, dtype: int64
Probability values
abandon     3.400000
abbrevy     6.333333
abdom       6.333333
abdomin     6.333333
aberdeen    6.333333
              ...   
zint        3.400000
zip         6.333333
zon         6.333333
zoolog      2.142857
zoophyt     3.400000
Length: 3772, dtype: float64


### Get Document Probablities

In [16]:
similarity = {}
similarity["query"] = {}
for column in df:
    ll = (df[df[column]>0].index).intersection(qw)
    similarity["query"][column]=prob[ll].product()

Index(['fish'], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index(['fish'], dtype='object')
Index([], dtype='object')
Index(['fish'], dtype='object')
Index(['fish', 'lung'], dtype='object')
Index([], dtype='object')
Index([], dtype='object')


### Output: Most relevant documents based on given query

In [11]:
result = pd.DataFrame(similarity).sort_values("query",ascending=False)
result

,query
T7.txt,9.148148
T1.txt,1.444444
T4.txt,1.444444
T6.txt,1.444444
T10.txt,1.000000
T2.txt,1.000000
T3.txt,1.000000
T5.txt,1.000000
T8.txt,1.000000
T9.txt,1.000000


### Checking correctness of ranking using inverted index

In [12]:
for word in qw:
    print("\n")
    if word in inverted_index.index:
        print(word)
        print(inverted_index.loc[word])
    else:
        print(word,"No entry in inverted index")
    print("\n")



fish
T1.txt     11
T10.txt     0
T2.txt      0
T3.txt      0
T4.txt      2
T5.txt      0
T6.txt      2
T7.txt     30
T8.txt      0
T9.txt      0
Name: fish, dtype: int64




lung
T1.txt     0
T10.txt    0
T2.txt     0
T3.txt     0
T4.txt     0
T5.txt     0
T6.txt     0
T7.txt     7
T8.txt     0
T9.txt     0
Name: lung, dtype: int64


